In [109]:
from __future__ import absolute_import
from __future__ import print_function
from collections import defaultdict
import os, sys
import xml.etree.ElementTree as et
import csv

import matplotlib
if 'matplotlib.backends' not in sys.modules:
    if 'TEXTTEST_SANDBOX' in os.environ or (os.name == 'posix' and 'DISPLAY' not in os.environ):
        matplotlib.use('Agg')
import matplotlib.pyplot as plt  # noqa
%matplotlib inline
import math  # noqa

In [110]:
# import sumolib
import os, sys
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:   
    sys.exit("please declare environment variable 'SUMO_HOME'")

from sumolib.xml import parse_fast_nested  # noqa
from sumolib.miscutils import uMin, uMax, parseTime  # noqa
from sumolib.options import ArgumentParser  # noqa

In [111]:
# reconstruct this part below
def fast_iter(context, func, args=[], kwargs={}):
    # http://www.ibm.com/developerworks/xml/library/x-hiperfparse/
    # Author: Liza Daly
    for event, elem in context:
        func(elem, *args, **kwargs)
        elem.clear()
        while elem.getprevious() is not None:
            del elem.getparent()[0]
    del context

In [112]:
input_file_prefix_list = [
#     ["yi2lppoa", "2022-10-03-15-05-50"], 
#     ["6q2jgjtg", "2022-10-03-22-08-05"]
    ["g4742en5", "2022-11-09-06-40-04"], 
    ["sztm_fuq", "2022-11-09-09-16-59"]# TAoI simple 1channel 100sec emission
    ["g4742en5", "2022-11-09-06-40-04"], 
    ["sztm_fuq", "2022-11-09-09-16-59"]# TAoI simple 1channel 100sec emission
]

scenario = 'ls'
# scenario = 'due.static'

In [113]:
input_file_list = []
for i in input_file_prefix_list:
    x = ["./Aircomp/sumo-launchd-tmp-" + i[0] + "/" + i[1] + "_" + scenario + ".collision.xml",
         "./Aircomp/sumo-launchd-tmp-" + i[0] + "/" + i[1] + "_" + scenario + ".tripinfo.xml", 
         "./Aircomp/sumo-launchd-tmp-" + i[0] + "/" + i[1] + "_" + scenario + ".summary.xml", 
    ]
    input_file_list.append(x)
collisions_list = []
tripinfos_list = []

In [114]:
# collision
# fast_iter 

for input_file in input_file_list:
    print("Current running: ", input_file[1])
    collisions = []
    tripinfos = []
    for event, elem in et.iterparse(input_file[1], events = ("end", )):
        if elem.tag == 'collision':
            cl = {
                'time': elem.get('time'), 
                'type': elem.get('type'), 
                'lane': elem.get('lane'), 
                'pos': elem.get('pos'), 
                'collider': elem.get('collider'), 
                'victim': elem.get('victim'), 
                'colliderType': elem.get('colliderType'), 
                'victimType': elem.get('victimType'), 
                'colliderSpeed': elem.get('colliderSpeed'), 
                'victimSpeed': elem.get('victimSpeed')
            }
        else:
            cl = {
                'time': "NA", 
                'type': "NA", 
                'lane': "NA", 
                'pos': "NA", 
                'collider': "NA", 
                'victim': "NA", 
                'colliderType': "NA", 
                'victimType': "NA", 
                'colliderSpeed': "NA", 
                'victimSpeed': "NA"
            }
            if elem.tag == 'tripinfo': 
                tp = {
                    'id': elem.get('id'), 
                    'depart': elem.get('depart'), 
                    'departSpeed': elem.get('departSpeed'),
                    'departDelay': elem.get('departDelay'), 
                    'arrival': elem.get('arrival'), 
                    'arrivalSpeed': elem.get('arrivalSpeed'), 
                    'duration': elem.get('duration'), 
                    'routeLength': elem.get('routeLength'), 
                    'waitingTime': elem.get('waitingTime'), 
                    'waitingCount': elem.get('waitingCount'), 
                    'stopTime': elem.get('stopTime'), 
                    'timeLoss': elem.get('timeLoss'), 
                    'rerouteNo': elem.get('rerouteNo'), 
                    'vType': elem.get('vType'), 
                    'speedFactor': elem.get('speedFactor'), 
                    'vaporized': elem.get('vaporized')
                }
                for child in elem:
                    if child.tag == 'emissions': 
                        tp['CO_abs'] = child.get('CO_abs')
                        tp['CO2_abs'] = child.get('CO2_abs')
                        tp['HC_abs'] = child.get('HC_abs')
                        tp['PMx_abs'] = child.get('PMx_abs')
                        tp['NOx_abs'] = child.get('NOx_abs')
                        tp['fuel_abs'] = child.get('fuel_abs')
                        tp['electricity_abs'] = child.get('electricity_abs')
                    else:
                        tp['CO_abs'] = "NA"
                        tp['CO2_abs'] = "NA"
                        tp['HC_abs'] = "NA"
                        tp['PMx_abs'] = "NA"
                        tp['NOx_abs'] = "NA"
                        tp['fuel_abs'] = "NA"
                        tp['electricity_abs'] = "NA"
#                 else:
#                     tp = {
#                         'id': 'NA', 
#                         'depart': 'NA', 
#                         'departSpeed': 'NA', 
#                         'departDelay': 'NA', 
#                         'arrival': "NA", 
#                         'arrivalSpeed': 'NA', 
#                         'duration': 'NA', 
#                         'routeLength': 'NA', 
#                         'waitingTime': 'NA', 
#                         'waitingCount': 'NA', 
#                         'stopTime': 'NA', 
#                         'timeLoss': 'NA', 
#                         'rerouteNo': 'NA', 
#                         'vType': 'NA', 
#                         'speedFactor': 'NA', 
#                         'vaporized': 'NA'
#                     }
#                     tp['CO_abs'] = "NA"
#                     tp['CO2_abs'] = "NA"
#                     tp['HC_abs'] = "NA"
#                     tp['PMx_abs'] = "NA"
#                     tp['NOx_abs'] = "NA"
#                     tp['fuel_abs'] = "NA"
#                     tp['electricity_abs'] = "NA"
        collisions.append(cl)
        tripinfos.append(tp)
    collisions_list.append(collisions)
    tripinfos_list.append(tripinfos)
#         while elem.getprevious() is not None:
#             del elem.getparent()[0]

Current running:  ./Aircomp/sumo-launchd-tmp-g4742en5/2022-11-09-06-40-04_ls.tripinfo.xml
Current running:  ./Aircomp/sumo-launchd-tmp-sztm_fuq/2022-11-09-09-16-59_ls.tripinfo.xml


In [115]:
# output data to csv file
a = 0
for inpu in input_file_prefix_list:
    csvfile = open("data-tripinfo-" + inpu[0] + ".csv","w",newline='')
    data_csv = csv.writer(csvfile)
    data_csv.writerow([
        'id', 'depart', 'departSpeed', 'departDelay', 
        'arrival', 'arrivalSpeed', 'duration', 'routeLength', 
        'waitingTime', 'waitingCount', 'stopTime', 
        'timeLoss', 'rerouteNo', 'vType', 
        'speedFactor', 'vaporized', 
        'CO_abs', 'CO2_abs', 'HC_abs', 
        'PMx_abs', 'NOx_abs', 
        'fuel_abs', 'electricity_abs'
    ])
    for i in range(0,len(tripinfos_list[a])):
        data_csv.writerow([
            tripinfos_list[a][i]['id'], tripinfos_list[a][i]['depart'], tripinfos_list[a][i]['departSpeed'], tripinfos_list[a][i]['departDelay'], 
            tripinfos_list[a][i]['arrival'],  tripinfos_list[a][i]['arrivalSpeed'], tripinfos_list[a][i]['duration'], tripinfos_list[a][i]['routeLength'], 
            tripinfos_list[a][i]['waitingTime'], tripinfos_list[a][i]['waitingCount'], tripinfos_list[a][i]['stopTime'], 
            tripinfos_list[a][i]['timeLoss'], tripinfos_list[a][i]['rerouteNo'], tripinfos_list[a][i]['vType'], 
            tripinfos_list[a][i]['speedFactor'], tripinfos_list[a][i]['vaporized'], 
            tripinfos_list[a][i]['CO_abs'], tripinfos_list[a][i]['CO2_abs'], tripinfos_list[a][i]['HC_abs'], 
            tripinfos_list[a][i]['PMx_abs'], tripinfos_list[a][i]['NOx_abs'], 
            tripinfos_list[a][i]['fuel_abs'], tripinfos_list[a][i]['electricity_abs'] 
        ])
    csvfile.close()
    a = a + 1

In [116]:
print(tripinfos_list)

[[{'id': '0-veh45.3', 'depart': '588.564', 'departSpeed': '0.000', 'departDelay': '588.564', 'arrival': '-1.000', 'arrivalSpeed': '-1.000', 'duration': '11.436', 'routeLength': '3.442', 'waitingTime': '1.877', 'waitingCount': '1', 'stopTime': '0.000', 'timeLoss': '11.177', 'rerouteNo': '0', 'vType': 'CarD@0-veh45.3', 'speedFactor': '0.962', 'vaporized': 'end', 'CO_abs': '1815.946155', 'CO2_abs': '29624.382157', 'HC_abs': '8.961666', 'PMx_abs': '0.725027', 'NOx_abs': '13.493646', 'fuel_abs': '9449.446029', 'electricity_abs': '0'}, {'id': '0-veh9', 'depart': '0.000', 'departSpeed': '0.000', 'departDelay': '0.000', 'arrival': '246.250', 'arrivalSpeed': '16.410', 'duration': '246.250', 'routeLength': '3995.720', 'waitingTime': '0.033', 'waitingCount': '1', 'stopTime': '0.000', 'timeLoss': '2.757', 'rerouteNo': '0', 'vType': 'CarA', 'speedFactor': '1.181', 'vaporized': '', 'CO_abs': '988.994671', 'CO2_abs': '675734.207247', 'HC_abs': '5.679845', 'PMx_abs': '5.119120', 'NOx_abs': '206.290365